## Property Sale Prices in London

#### Data Download

In [ ]:
# Price paid data download:
!wget -P data http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.csv

In [ ]:
# EPC data dowbloaded from
# https://epc.opendatacommunities.org/domestic/search
# Registration required.

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import requests
import os
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

## Price Paid Data Dictionary

Web-scaped from
https://www.gov.uk/guidance/about-the-price-paid-data#explanations-of-column-headers-in-the-ppd


In [2]:
url = "https://www.gov.uk/guidance/about-the-price-paid-data#explanations-of-column-headers-in-the-ppd"
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

element = soup.find('table')

rows = element.find_all('tr')

headers = [header.text for header in rows[0].find_all('th')]

data = []
for row in rows[1:]:
    data.append([col.text for col in row.find_all('td')])

col1data = []
col2data = []
for row in rows[1:]:
    col1data.append([col.text for col in row.find_all('th')])
    col2data.append([col.text for col in row.find_all('td')])

dat1 = pd.DataFrame(col1data, columns=headers[:1])
dat2 = pd.DataFrame(col2data, columns=headers[1:])

pp_dictionary = pd.concat([dat1,dat2],axis=1)

pp_dictionary

,Data item,Explanation (where appropriate)
0,Transaction unique identifier,A reference number which is generated automati...
1,Price,Sale price stated on the transfer deed.
2,Date of Transfer,"Date when the sale was completed, as stated on..."
3,Postcode,This is the postcode used at the time of the o...
4,Property Type,"D = Detached, S = Semi-Detached, T = Terraced,..."
5,Old/New,Indicates the age of the property and applies ...
6,Duration,"Relates to the tenure: F = Freehold, L= Leaseh..."
7,PAON,Primary Addressable Object Name. Typically the...
8,SAON,Secondary Addressable Object Name. Where a pro...
9,Street,


## Price Paid Data

In [3]:
# price paid data
PP_FILE_PATH ="data/pp-complete.csv"

In [4]:
spark = SparkSession.builder.appName('housesalesdata').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/25 11:28:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
pp = spark.read.csv(PP_FILE_PATH, inferSchema=True, header=False)

In [6]:
headers = ['TRANSACTION_UNIQUE_IDENTIFIER','PRICE',
           'DATE_OF_TRANSFER','POSTCODE','PROPERTY_TYPE',
           'OLD_NEW','DURATION','PAON','SAON','STREET',
           'LOCALITY','TOWN_CITY','DISTRICT','COUNTY',
           'PPD_CATEGORY_TYPE','RECORD_STATUS_MONTHLY_FILE_ONLY']
pp = pp.toDF(*headers)

pp.limit(5).toPandas()

,TRANSACTION_UNIQUE_IDENTIFIER,PRICE,DATE_OF_TRANSFER,POSTCODE,PROPERTY_TYPE,OLD_NEW,DURATION,PAON,SAON,STREET,LOCALITY,TOWN_CITY,DISTRICT,COUNTY,PPD_CATEGORY_TYPE,RECORD_STATUS_MONTHLY_FILE_ONLY
0,{F887F88E-7D15-4415-804E-52EAC2F10958},70000,1995-07-07,MK15 9HP,D,N,F,31,None,ALDRICH DRIVE,WILLEN,MILTON KEYNES,MILTON KEYNES,MILTON KEYNES,A,A
1,{40FD4DF2-5362-407C-92BC-566E2CCE89E9},44500,1995-02-03,SR6 0AQ,T,N,F,50,None,HOWICK PARK,SUNDERLAND,SUNDERLAND,SUNDERLAND,TYNE AND WEAR,A,A
2,{7A99F89E-7D81-4E45-ABD5-566E49A045EA},56500,1995-01-13,CO6 1SQ,T,N,F,19,None,BRICK KILN CLOSE,COGGESHALL,COLCHESTER,BRAINTREE,ESSEX,A,A
3,{28225260-E61C-4E57-8B56-566E5285B1C1},58000,1995-07-28,B90 4TG,T,N,F,37,None,RAINSBROOK DRIVE,SHIRLEY,SOLIHULL,SOLIHULL,WEST MIDLANDS,A,A
4,{444D34D7-9BA6-43A7-B695-4F48980E0176},51000,1995-06-28,DY5 1SA,S,N,F,59,None,MERRY HILL,BRIERLEY HILL,BRIERLEY HILL,DUDLEY,WEST MIDLANDS,A,A


In [7]:
pp.select(
    F.min('DATE_OF_TRANSFER').alias('min_date'),
    F.max('DATE_OF_TRANSFER').alias('max_date')
         ).show()

+-------------------+-------------------+
|           min_date|           max_date|
+-------------------+-------------------+
|1995-01-01 00:00:00|2023-09-29 00:00:00|
+-------------------+-------------------+



In [8]:
pp_london = pp.filter(F.col('COUNTY')=='GREATER LONDON')\
    .filter(F.col('DATE_OF_TRANSFER')>='2010-01-01')

In [9]:
pp_london.select(F.count("*")).show()

+--------+
|count(1)|
+--------+
| 1494250|
+--------+



In [10]:
pp_london.filter(F.col("POSTCODE").isNull()).count()

4623

In [11]:
pp_london = pp_london.filter(F.col("POSTCODE").isNotNull())

In [12]:
pp_london = pp_london.toPandas()

## Load EPC data

---- Explanation on use of EPC data for floor area
---- match

In [13]:
districts = pd.read_csv("data/Local_Authority_Districts_(April_2023)_Names_and_Codes_in_the_United_Kingdom.csv")
lon_boroughs = pd.read_csv("data/london_borough.csv") # from Wikipedia

lon_boroughs = lon_boroughs.rename(columns={"Borough":"LAD23NM"})

lon_districts = lon_boroughs.merge(districts[["LAD23NM","LAD23CD"]],
                  how="left",
                  on="LAD23NM")

lon_districts_set = set(lon_districts["LAD23CD"])

In [36]:
fol = 'epc_data/'
lista = os.listdir(fol)

In [ ]:
# pr = pd.read_csv(fol+"domestic-E09000019-Islington/certificates.csv")

In [ ]:
# pr1 = pr[["ADDRESS","LODGEMENT_DATE","TOTAL_FLOOR_AREA","LODGEMENT_DATETIME"]]

In [ ]:
# pr1 = pr1.merge(pr1["ADDRESS"].value_counts().rename('count'),
#           how='left',
#           # left_on="ADDRESS",
#           right_index=True)

In [15]:
# pr1[pr1['count']>1].sort_values(["count","ADDRESS","LODGEMENT_DATETIME"], ascending=False)

In [37]:
cols_epc = ["ADDRESS","POSTCODE","TOTAL_FLOOR_AREA"]

dfs = []
for fol1 in lista:
    dist = fol1.split("-")[1]
    if dist in lon_districts_set:
        tmp = pd.read_csv(fol+fol1+"/certificates.csv",
                          usecols = cols_epc,
                          low_memory=False)
        dfs.append(tmp)

In [38]:
london_epc = pd.concat(dfs, ignore_index=True)

### Reformat columns and merge

In [18]:
def transform(df, cols:list):
    transformed_cols = []
    for c in cols:
        newc = c+"_fmt"
        df[newc] = df[c].str.strip()
        df[newc] = df[newc].str.lower()
        df[newc] = df[newc].str.replace(" ","")
        df[newc] = df[newc].str.replace(",","")
        transformed_cols.append(newc)
    return transformed_cols

In [19]:
pp_cols = ["SAON", "PAON", "STREET","POSTCODE"]

transformed_cols_pp = transform(pp_london, pp_cols)

In [20]:
pp_london["address_match_string"] = pp_london[transformed_cols_pp].fillna("").astype(str).agg(lambda x:''.join(x), axis=1)

In [39]:
epc_cols = ['ADDRESS', 'POSTCODE']
transformed_cols_epc = transform(london_epc, epc_cols)

In [40]:
london_epc["address_match_string"] = london_epc[transformed_cols_epc].fillna("").astype(str).agg(lambda x:''.join(x), axis=1)

## Cleaning london_epc

For the addresses that have multiple values for "TOTAL_FLOOR_AREA", I keep only the highest.

In [41]:
london_epc["address_match_string"].value_counts()

benthamhouse2-4stationwayse186fj              33
47elizabethgardensha74ug                      24
flat2busheycourthazelroadda82ln               23
82dgreenwrythelanesm52dp                      22
20barnettcloseda82eg                          22
                                              ..
brierleyhousebarnetroaden53lw                  1
10grantsclosenw71dd                            1
flat12ashmorecourt274colneyhatchlanen113dq     1
flat214thegrovenw119sh                         1
2barringtoncourt29jeffreysroadsw46qu           1
Name: address_match_string, Length: 2919643, dtype: int64

In [42]:
london_epc = london_epc.sort_values(["address_match_string","TOTAL_FLOOR_AREA"], ascending=False)\
    .drop_duplicates("address_match_string",keep='first')

In [43]:
london_epc["address_match_string"].value_counts()

zinniastanmoreroadn153pr                          1
38miltonavenuenw108eu                             1
38minetdriveub33jn                                1
38minervacloselondonsw96nz                        1
38minardroadse61nr                                1
                                                 ..
flat1307100ageorgestreetcr01gp                    1
flat1306wharfsidepointsouth4prestonsroade149ex    1
flat1306wandsworthbridgeroadsw62ua                1
flat1306vantagepoint2junctionroadn195ff           1
!3aheathroadtw32nj                                1
Name: address_match_string, Length: 2919643, dtype: int64

In [ ]:
# london_epc = london_epc.merge(london_epc["address_match_string"].value_counts().rename("count"),
#                  how='left',
#                  left_on="address_match_string",
#                  right_index=True)

In [ ]:
# london_epc[london_epc["count"]>1].sort_values("address_match_string")

### Merging

In [44]:
len(pp_london)

1489627

In [45]:
data_london = pp_london.merge(london_epc[["address_match_string", "TOTAL_FLOOR_AREA"]],
               how = 'left',
               on = "address_match_string")

In [46]:
len(data_london)

1489627

In [48]:
match_rate = (len(data_london[data_london["TOTAL_FLOOR_AREA"].notna()])/len(data_london))

In [49]:
print(f'Match Rate: {round(match_rate*100)}%')

Match Rate: 80%


In [59]:
data_london = data_london.drop(transformed_cols_pp+["address_match_string"],axis=1)

In [66]:
data_london["PARTIAL_POSTCODE"] = data_london["POSTCODE"].str.split(expand=True)[0]

In [67]:
data_london.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1489627 entries, 0 to 1489626
Data columns (total 18 columns):
 #   Column                           Non-Null Count    Dtype         
---  ------                           --------------    -----         
 0   TRANSACTION_UNIQUE_IDENTIFIER    1489627 non-null  object        
 1   PRICE                            1489627 non-null  int32         
 2   DATE_OF_TRANSFER                 1489627 non-null  datetime64[ns]
 3   POSTCODE                         1489627 non-null  object        
 4   PROPERTY_TYPE                    1489627 non-null  object        
 5   OLD_NEW                          1489627 non-null  object        
 6   DURATION                         1489627 non-null  object        
 7   PAON                             1489621 non-null  object        
 8   SAON                             559561 non-null   object        
 9   STREET                           1489570 non-null  object        
 10  LOCALITY                      

In [79]:
pp_dictionary

,Data item,Explanation (where appropriate)
0,Transaction unique identifier,A reference number which is generated automati...
1,Price,Sale price stated on the transfer deed.
2,Date of Transfer,"Date when the sale was completed, as stated on..."
3,Postcode,This is the postcode used at the time of the o...
4,Property Type,"D = Detached, S = Semi-Detached, T = Terraced,..."
5,Old/New,Indicates the age of the property and applies ...
6,Duration,"Relates to the tenure: F = Freehold, L= Leaseh..."
7,PAON,Primary Addressable Object Name. Typically the...
8,SAON,Secondary Addressable Object Name. Where a pro...
9,Street,


In [88]:
pp_dictionary[pp_dictionary["Data item"]=="Record Status - monthly file only"]["Explanation (where appropriate)"].values

array(['Indicates additions, changes and deletions to the records.(see guide below).A = AdditionC = ChangeD = DeleteNote that where a transaction changes category type due to misallocation (as above) it will be deleted from the original category type and added to the correct category with a new transaction unique identifier.'],
      dtype=object)

In [73]:
PROPERTY_TYPE_labels = {
    "D" : "Detached",
    "S" : "Semi-Detached",
    "T" : "Terraced",
    "F" : "Flats/Maisonettes",
    "O" : "Other"
}

data_london["PROPERTY_TYPE"] = data_london["PROPERTY_TYPE"].map(PROPERTY_TYPE_labels)

In [81]:
old_new_labels = {
    "Y":"newly built",
    "N":"old"
}
data_london["OLD_NEW"] = data_london["OLD_NEW"].map(old_new_labels )

In [85]:
duration_labels = {
    "F" : "Freehold", 
    "L" : "Leasehold"
}

data_london["DURATION"] = data_london["DURATION"].map(duration_labels)

In [94]:
# data_london.to_parquet("data/london.gzip")
data_london.to_csv("data/london.csv", index=None)

In [92]:
spark.stop()

In [103]:
import pandas as pd

In [104]:
df = pd.read_csv("data/london.csv")

In [105]:
df.head()

,TRANSACTION_UNIQUE_IDENTIFIER,PRICE,DATE_OF_TRANSFER,POSTCODE,PROPERTY_TYPE,OLD_NEW,DURATION,PAON,SAON,STREET,LOCALITY,TOWN_CITY,DISTRICT,COUNTY,PPD_CATEGORY_TYPE,RECORD_STATUS_MONTHLY_FILE_ONLY,TOTAL_FLOOR_AREA,PARTIAL_POSTCODE
0,{2A83B3D0-0734-420D-B48E-1CB8F384DA98},250000,2010-02-03,SE10 8AT,Flats/Maisonettes,old,Leasehold,DARTMOUTH COURT,FLAT 70,DARTMOUTH GROVE,NaN,LONDON,LEWISHAM,GREATER LONDON,A,A,69.20,SE10
1,{E88ACCD1-6563-44AE-B78A-192A8100FF32},405000,2010-03-01,SW11 2PB,Semi-Detached,old,Freehold,26,NaN,MOSSBURY ROAD,NaN,LONDON,WANDSWORTH,GREATER LONDON,A,A,48.75,SW11
2,{EFAA44F3-7A6E-45BD-A48B-192AA913F3E7},575000,2010-06-11,SE1 7HD,Flats/Maisonettes,old,Leasehold,9,FLAT 73,ALBERT EMBANKMENT,NaN,LONDON,LAMBETH,GREATER LONDON,A,A,72.00,SE1
3,{EB677C10-9596-45B9-817B-1590C0AFC424},450000,2010-01-15,E1 6RL,Flats/Maisonettes,newly built,Leasehold,110,FLAT 2,BRICK LANE,NaN,LONDON,TOWER HAMLETS,GREATER LONDON,A,A,100.00,E1
4,{5D363FE5-353E-4385-B8EB-23E024D74DE8},315000,2010-05-25,SE6 1AL,Terraced,old,Freehold,341,NaN,BROWNHILL ROAD,NaN,LONDON,LEWISHAM,GREATER LONDON,A,A,107.00,SE6


In [106]:
sam = df.sample(100)

In [110]:
sam["DATE_OF_TRANSFER"].dtype

dtype('O')

In [121]:
df["DATE_OF_TRANSFER"] = pd.to_datetime(df["DATE_OF_TRANSFER"])

In [112]:
from datetime import datetime

In [129]:
new = df[(df["DATE_OF_TRANSFER"]>datetime(2023,4,1))
  &(df["OLD_NEW"]=="newly built")]

In [133]:
old = df[(df["DATE_OF_TRANSFER"]>datetime(2023,4,1))
  &(df["OLD_NEW"]=="old")].sample(100)

In [134]:
pr = pd.concat([new, old])

In [136]:
pr["OLD_NEW"].value_counts()

newly built    104
old            100
Name: OLD_NEW, dtype: int64

In [137]:
pr.to_csv("prova.csv", index=None)

In [157]:
pr[(pr["DATE_OF_TRANSFER"] >='2023-07-01')
  &(pr["DATE_OF_TRANSFER"] <'2023-09-01')]\
    ["OLD_NEW"].value_counts()

old            38
newly built    10
Name: OLD_NEW, dtype: int64

In [152]:
pr["DATE_OF_TRANSFER"].value_counts().to_csv()

2023-04-06    11
2023-06-01     9
2023-04-11     7
2023-06-30     7
2023-07-31     7
              ..
2023-09-01     1
2023-07-03     1
2023-08-18     1
2023-04-12     1
2023-06-28     1
Name: DATE_OF_TRANSFER, Length: 81, dtype: int64